## 第一階段：取得所有必要的職缺網址

In [16]:
from datetime import datetime
import json
import random
import re
import time
from urllib.parse import unquote, quote

from bs4 import BeautifulSoup
import requests

In [140]:
search_url = input('請輸入搜尋頁面網址 ')

請輸入搜尋頁面網址 https://www.104.com.tw/jobs/search/?ro=0&keyword=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB&area=6001002000%2C6001001000&jobsource=2018indexpoc


In [86]:
kw = input('請輸入關鍵字 ')
search_url = \
f'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={quote(kw)}&area=\
6001002000%2C6001001000&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc'

請輸入關鍵字 資料工程師


In [48]:
def url_to_soup(url):
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    resp = requests.get(url, headers=head)
    soup = BeautifulSoup(resp.text)
    return soup

In [19]:
def get_page_urls(search_url):
    soup = url_to_soup(search_url)
    jobs = soup.select('.js-job-link')
    page_urls = []
    for job in jobs:
        try:
            page_urls.append('https:' + job['href'].split('&')[0])
        except:
            page_urls.append('https:' + job['href'])
    return page_urls

In [22]:
def get_all_urls(search_url):
    job_urls = []
    for page in range(1, 151):
        if 'page=' in search_url:
#           用 f-string 就不必使用 str(page)了，因為page必轉為str
            search_url = re.sub(r'page=\d*', f'page={page}',
                                search_url)
        else:
            search_url = search_url + f'&page={page}'
            
        print(f'{page}頁', end='\t')
        
        try:
            page_urls = get_page_urls(search_url)
        except BaseException as err:
            print(err)
            break
            
        job_urls.extend(page_urls)
        
        if len(page_urls) < 20:
            break
        time.sleep(random.uniform(4, 10))

    print('')
    print('總共收集', len(job_urls), '個職缺網址')
    print('完成')
    return job_urls

In [87]:
result = get_all_urls(search_url)

1頁	2頁	3頁	4頁	5頁	6頁	7頁	8頁	9頁	10頁	11頁	12頁	13頁	14頁	15頁	16頁	17頁	18頁	19頁	20頁	21頁	22頁	23頁	24頁	25頁	26頁	27頁	28頁	29頁	30頁	31頁	32頁	33頁	34頁	35頁	36頁	37頁	38頁	39頁	40頁	41頁	42頁	43頁	44頁	45頁	46頁	47頁	48頁	49頁	50頁	51頁	52頁	53頁	54頁	55頁	56頁	57頁	58頁	59頁	60頁	61頁	62頁	63頁	64頁	65頁	66頁	67頁	68頁	69頁	70頁	71頁	72頁	73頁	74頁	75頁	76頁	77頁	78頁	79頁	80頁	81頁	82頁	83頁	84頁	85頁	86頁	87頁	88頁	89頁	90頁	91頁	92頁	93頁	94頁	95頁	96頁	97頁	98頁	99頁	100頁	101頁	102頁	103頁	104頁	105頁	106頁	107頁	108頁	109頁	110頁	111頁	112頁	113頁	114頁	115頁	116頁	117頁	118頁	119頁	120頁	121頁	122頁	123頁	124頁	125頁	126頁	127頁	128頁	129頁	130頁	131頁	132頁	133頁	134頁	135頁	136頁	137頁	138頁	139頁	140頁	141頁	142頁	143頁	144頁	145頁	146頁	147頁	148頁	149頁	150頁	
總共收集 3010 個職缺網址
完成


### 將結果寫入JSON檔

#### 先從網址取出關鍵字

In [88]:
keyword_raw = re.search(r'keyword=[^&]*', search_url).group().split('=')[1]
keyword = unquote(keyword_raw)
keyword

'資料工程師'

#### 記錄當下時間，以指定格式

In [89]:
timestamp = datetime.now().strftime('%m%d-%H%M')
timestamp

'0429-0951'

In [90]:
len(result)

3010

In [91]:
result_set = set(result)
len(result_set)

2848

In [92]:
result_set.discard('https:javascript:void(0);')
result_tuple = tuple(result_set)
len(result_tuple)

2847

In [93]:
with open(f'job_urls/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(result_tuple, f)

### (選填)從檔案取得job_urls與關鍵字

In [36]:
fromfile = input('請輸入檔名 ')
with open(f'job_urls/{fromfile}.json', encoding='utf8') as file:
    result_tuple = json.load(file)
keyword = fromfile.split('-')[0]
len(result_tuple)

請輸入檔名 資料工程師-0428-0059


2995

### 要排除重複的job_urls檔

In [94]:
filename = input('請輸入檔名 ')
with open(f'job_urls/{filename}.json', encoding='utf8') as file:
    urls = json.load(file)
len(urls)

請輸入檔名 data engineer-0428-2051×Python-0428-0245×後端工程師-0428-2020


5835

### 合併複數job_urls檔

In [81]:
filename1 = input('請輸入檔名1：')
with open(f'job_urls/{filename1}.json', encoding='utf8') as file:
    urls1 = json.load(file)
len(urls1)

請輸入檔名1：data engineer-0428-2051


3001

In [82]:
filename2 = input('請輸入檔名2：')
with open(f'job_urls/{filename2}.json', encoding='utf8') as file:
    urls2 = json.load(file)
len(urls2)

請輸入檔名2：Python-0428-0245×後端工程師-0428-2020


3524

In [13]:
filename3 = input('請輸入檔名3：')
with open(f'job_urls/{filename3}.json', encoding='utf8') as file:
    urls3 = json.load(file)
len(urls3)

請輸入檔名3：後端工程師-0425-2321


1503

In [14]:
urls1.extend(urls2)
urls1.extend(urls3)
len(urls1)

6752

In [83]:
urls1.extend(urls2)
len(urls1)

6525

In [84]:
urls_tuple = tuple(set(urls1))
len(urls_tuple)

5835

In [45]:
with open(f'job_urls/{filename1}×{filename2}×{filename3}.json', 'w') as f:
    json.dump(urls_tuple, f)

In [85]:
with open(f'job_urls/{filename1}×{filename2}.json', 'w') as f:
    json.dump(urls_tuple, f)

In [95]:
block = ('前端', '測試', '分析', 'front', 'php', 'java', 'asp', 'c#', 'ios', 
        'android', 'react', 'vue', 'angular', 'ux', 'ui', 'rail',
        'ruby', '.net', 'core', 'test', '視覺', '資安', '網頁設計', 'html',
        'css', 'jquery', '影像', '行政', 'manager', '韌體', 'c+', 'unity',
        '硬體', '介面', 'qa', 'quality', '維護', '研發替代役', 'designer',
        '美術', '物聯網', 'iot', '介面設計', '嵌入', 'devops', 'scientist',
        '人工智', '圖像', '算法', 'hardware', '硬體', '品質', 'lte', '實習',
        '科學家', 'firmware', 'sre', 'reliability', '可靠性', 'embed', '顧問',
        'vr', '助理', '自動駕', '深度學習', 'nlp', '工讀', 'intern', '兼職',
        '襄理', '副理', '經理', 'fae', 'bot', 'crm', '主管', '主任', 'analyst',
        '網管', 'integr', 'pm', '3d', '動畫', '行銷', '研究員', '機率', '特效',
        'mis', '光學', '約聘', '總監', 'algorithm','檢測', '支援', '客服', '製作人',
        'erp', '網站設計', '編輯', '專員', '總監', '小編', '機器人', '企劃', 'node',
        '補習班', '證券', '老師', '自動化', '機器學習', '講師', 'learning', '教師',
        '交易員', '驗證', '汽車', '財務工程', '傳銷', '業務', '平面設計', '美編',
        'dba', '譯者', '人壽', '期貨', '安全', '助教', 'ic設計', 'app ')

# app(只用加1個空隔) 遊戲 bi

In [96]:
def urls_to_data(result, block, block_urls):
    print(datetime.now().strftime('%H:%M:%S'))
    print(f'開始處理，總共有 {len(result)} 個項目')
    block_list = block
    data = []
    item = 1
    number = 1
    for url in result:
        blocked = False
        job = {}
        
        if url in block_urls:
            print('◆', end='\t')
            number = number + 1
            continue
        
        try:
            soup = url_to_soup(url)
            job['資料編號'] = f'第 {item} 項'
            job['職位網址'] = f' {url} '            
            job['職位名稱'] = soup.find_all('meta')[4]['content'].rstrip(
                ' - 104人力銀行')
            
            if 'python' in job['職位名稱'].lower():
                pass
            else:
#                 blocked = any(( item for item in block_list 
#                                if item in job['職位名稱'].lower()))
                for term in block_list:
                    if term in job['職位名稱'].lower():
                        blocked = True
                        break
            if blocked:
                print('×', end='\t')
                number = number + 1
                time.sleep(random.uniform(3, 9))
                continue

            job['工作地點'] = soup.find('dd', 'addr').text.strip().split()[0]
            job['工作內容'] = soup.find('p').text.replace('\r', '').replace(
                '\t', '').replace('\u3000', '').replace('\xa0', '')
            job['工作內容'] = job['工作內容'][:280]
            
            job['擅長工具'] = soup.select('.info')[1].find_all('dd')[-3].text
            job['工作技能'] = soup.select('.info')[1].find_all('dd')[-2].text
            job['其它條件'] = soup.select('.info')[1].find_all(
                'dd')[-1].text.replace('\r', '').replace('\t', '').replace(
                '\u3000', '').replace('\xa0', '')
            job['其它條件'] = job['其它條件'][:280]
            
        except:
            print('××', end='\t')
            number = number + 1
            time.sleep(random.uniform(3, 9))
            continue

        for k, v in job.items():
            if v in ('不拘', '未填寫'):
                job[k] = ''
        data.append(job)
        print(number, end='\t')
        item = item + 1
        number = number + 1
        time.sleep(random.uniform(3, 9))
    print('')
    print(f'完成，實際抓取 {len(data)} 個項目')
    print(datetime.now().strftime('%H:%M:%S'))
    return data

In [98]:
data = urls_to_data(result_tuple, block=block, block_urls=urls)

09:52:35
開始處理，總共有 2847 個項目
1	◆	◆	4	◆	◆	7	◆	9	10	×	×	13	14	15	×	◆	×	×	20	×	×	×	×	×	×	◆	×	×	×	31	32	◆	×	35	×	◆	◆	×	40	41	×	×	×	×	46	◆	×	49	50	×	×	×	◆	55	◆	×	◆	×	◆	×	◆	×	64	×	×	67	◆	×	◆	×	×	73	×	×	×	×	78	79	×	81	×	83	×	◆	◆	×	◆	◆	×	×	92	93	94	×	×	×	98	99	×	×	◆	×	×	105	106	×	×	109	×	◆	112	×	◆	◆	116	×	×	◆	×	×	◆	◆	◆	×	126	127	×	×	×	×	◆	◆	×	×	×	137	◆	×	×	×	×	×	×	×	◆	×	◆	149	×	151	152	◆	◆	×	156	157	◆	×	×	×	162	×	164	◆	166	◆	168	169	×	◆	172	◆	174	◆	◆	177	◆	179	×	◆	◆	183	×	185	×	×	×	◆	×	191	×	×	◆	195	×	197	×	199	◆	×	◆	203	◆	×	×	207	×	◆	210	211	×	×	×	◆	◆	217	×	◆	×	◆	222	223	◆	225	226	×	◆	◆	230	231	×	233	×	235	236	◆	×	×	240	◆	×	243	×	×	◆	247	248	×	×	×	◆	◆	◆	255	◆	×	×	259	260	261	262	◆	×	×	×	×	×	×	270	×	×	×	×	◆	×	277	◆	×	◆	×	◆	283	×	×	×	×	288	289	×	291	◆	◆	×	×	×	297	298	×	×	◆	◆	×	◆	◆	◆	×	×	×	×	◆	312	◆	×	×	◆	◆	318	×	×	321	322	◆	◆	325	326	×	328	◆	◆	331	332	×	334	335	336	×	×	×	340	×	×	×	◆	×	×	×	348	×	350	351	352	◆	×	355	356	357	358	◆	◆	361	×	◆	◆	◆	×	367	368	×	◆	×	372	◆	374	×	×	◆	◆	◆	×	◆	◆	×	×	◆	386	◆	3

In [61]:
data

[{'資料編號': '第 1 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=6l7zj ',
  '職位名稱': '大數據資料庫工程師 | 東方線上股份有限公司 | 台北市大安區',
  '工作地點': '台北市大安區信義路四段306號7樓',
  '工作內容': '1. 資料庫資料維護、處理。2. 資料庫備份、異地備援機制維護。3. 資料庫後端程式分析開發。4. 輔助與支援行銷研究業務。',
  '擅長工具': '伺服器網站管理維護、作業系統基本操作、資料備份與復原、資料庫系統管理維護、資料庫程式設計、資料庫軟體應用',
  '工作技能': 'MySQL CMA',
  '其它條件': '1. 對工作細心、認真負責、邏輯佳。2. 對大數據/巨量分析有熱忱者。3. 熟資料庫SQL 語法。4. 熟悉MySQL資料庫安裝、設定及操作實務經驗2年(含)以上。5. 具有MySQL CMA證照優先。6. 對消費性商品有市場敏感度者佳。'},
 {'資料編號': '第 2 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=6ddcs ',
  '職位名稱': '軟體專案管理師 | 大鵬科技股份有限公司 | 台北市內湖區',
  '工作地點': '台北市內湖區新湖二路258號',
  '工作內容': '負責Smart Home/IoT系統專案的規劃與執行，包含撰寫專案開發文件與規格書，管控專案進度、執行與追蹤檢討，跨部門協調與UX/UI/開發工程師/軟體測試工程師緊密協作，製作管理報表與相關文件整理歸檔。 負責Smart Home系統測試工作的計畫與執行，包含App, Web, Mobile Web以及未來可能出現的各種前端平台與後端系統；必須掌握軟體設計規格，與開發工程師/UX/UI緊密協作，規劃test plan，與test case，實際進行測試項目的執行與管理。 應徵條件：邏輯清晰，具備基礎網路知識與文書處理能力熟悉專案規劃及執行進度管理熟悉test plan規劃， test cas',
  '擅長工具': '',
  '工作技能': '',
  '其它條件': '應徵條件：邏輯清晰，具備基礎網路知識與文書處理能力熟悉專案規劃及執行進度管理熟悉test plan規劃， 

In [99]:
timestamp = datetime.now().strftime('%m%d-%H%M')

with open(f'data/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(data, f)

In [100]:
from collections import Counter

In [101]:
Counter(i['工作地點'][:6] for i in data).most_common()

[('台北市內湖區', 148),
 ('台北市中山區', 90),
 ('台北市松山區', 58),
 ('台北市大安區', 57),
 ('新北市中和區', 47),
 ('台北市中正區', 42),
 ('新北市三重區', 39),
 ('新北市汐止區', 39),
 ('台北市信義區', 37),
 ('新北市土城區', 27),
 ('新北市新店區', 27),
 ('台北市南港區', 25),
 ('新北市板橋區', 24),
 ('台北市大同區', 24),
 ('新北市五股區', 23),
 ('新北市新莊區', 19),
 ('新北市樹林區', 12),
 ('台北市萬華區', 9),
 ('新北市林口區', 8),
 ('台北市士林區', 6),
 ('新北市永和區', 6),
 ('新北市淡水區', 5),
 ('新北市蘆洲區', 4),
 ('台北市文山區', 4),
 ('台北市北投區', 4),
 ('新北市三峽區', 3),
 ('新北市鶯歌區', 3),
 ('新北市泰山區', 3),
 ('新北市瑞芳區', 2),
 ('新北市石碇區', 1),
 ('新北市深坑區', 1),
 ('新北市八里區', 1),
 ('新北市三芝區', 1)]

In [21]:
import json

In [24]:
json.dumps(aa)

'[1, 2, 3]'

In [ ]:
with open('data.json', 'w') as f:
    json.dump(aa, f)